This file includes a simple use case of the module.

A timed automata system is read from an xml file, an edge constraint is changed and the system is written back.

In [3]:
from pyutap import *

system = UTAP.TimedAutomataSystem() # Create a system
parseXMLFile("examples/literature/CAS.xml", system, True) # Read system from file, 0 indicates success.

0

In [4]:
templates = system.getTemplates() # Getting templates of the system

states_with_invariants = []

for template in templates:
        for state in template.states:
                if state.invariant.getSize() > 0: # Access states which have an invariant
                        states_with_invariants.append(state)
                        print("State uid:", state.uid) # Get state name
                        print("Invariant:", state.invariant) # Get state invariant
                        # print(state.toString()) # Print state in a readable form

State uid: (location) l3
Invariant: 1 && c <= 20
State uid: (location) l4
Invariant: 1 && g <= 0
State uid: (location) l5
Invariant: 1 && d <= 0
State uid: (location) l7
Invariant: 1 && f <= 0
State uid: (location) l8
Invariant: 1 && e <= 300
State uid: (location) l10
Invariant: 1 && e <= 0
State uid: (location) l11
Invariant: 1 && e <= 0
State uid: (location) l12
Invariant: 1 && e <= 0
State uid: (location) l13
Invariant: 1 && e <= 30
State uid: (location) l14
Invariant: 1 && g <= 0
State uid: (location) l15
Invariant: 1 && e <= 300


In [7]:
# Getting the edge guards of the system
Constants = UTAP.Constants

edge_guards_with_clock = []
for template in templates:
    for edge in template.edges:
        if (edge.guard.usesClock()): # Add guards that uses clocks to a list
            print("Source:", edge.src.uid, "; Dest:", edge.dst.uid)
            if edge.sync.getSize() > 0:
                print("Edge sync:", edge.sync)

            if edge.assign.getSize() > 0:
                print("Edge assign:", edge.assign)
                
            edge_guards_with_clock.append(edge.guard)

print(edge_guards_with_clock[0])
if edge_guards_with_clock[0].getKind() == Constants.AND:
    print("Guard has kind AND")

# Accessing subexpressions of expressions
print("Guard has", edge_guards_with_clock[0].getSize(), "subexpressions")
for i in range(edge_guards_with_clock[0].getSize()):
    print("Subexpression", i, edge_guards_with_clock[0][i])

Source: (location) l15 ; Dest: (location) l8
Edge sync: soundOff!
Source: (location) l13 ; Dest: (location) l14
Edge sync: unlock?
Edge assign: g = 0
Source: (location) l13 ; Dest: (location) l15
Edge sync: soundOff!
Source: (location) l3 ; Dest: (location) l6
Edge sync: armedOn!
e <= 100 && e >= 300
Guard has kind AND
Guard has 2 subexpressions
Subexpression 0 e <= 100
Subexpression 1 e >= 300


In [8]:
# Changing expression values
position = edge_guards_with_clock[0][0][1].getPosition() # Position of the expression
t = edge_guards_with_clock[0][0][1] # Sub-subexpression of the guard expression
if t.getKind() == Constants.CONSTANT:
    print("Const. old t=" + str(t.getValue()))
    #print("Double old t=" + str(t.getDoubleValue()))

exp = t.createConstant(100, position)
#exp = t.createDouble(100, position) # Creating double type expression
edge_guards_with_clock[0][0][1] = exp
if t.getKind() == Constants.CONSTANT:
    print("Const. new t=" + str(t.getValue()))
    #print("Double new t=" + str(t.getDoubleValue()))

Const. old t=100
Const. new t=100


In [9]:
from pyutap.path_analysis import *

path = []
for template in templates:
        print(template.edges[0].src.uid, template.edges[0].dst.uid)
        path.append(template.edges[0])

print(find_used_clocks(path))
print(path_exists(path))

(location) id2 (location) id1
(location) id3 (location) id7
(location) l7 (location) l6
(location) id9 (location) id8
(location) id11 (location) id10
{b'f'}
False


In [ ]:
writeXMLFile("example.xml", system) # Write the system back to a file, 0 indicates success.